# Build the model from first principles

In [1]:
from gba import Builder, Model, Protein, Metabolite, Reaction
from gba import SpeciesLocation, ReactionType, ReactionDirection

builder = Builder(name="toy")

### Add model information (ODS sheet 'Info')
builder.add_info(category="General", key="Name", content="toy")
builder.add_info(category="General", key="Description", content="Toy model")

### Create and add proteins (one protein per enzyme per reaction)
### - Masses in Da.
p1 = Protein(id="p1", mass=1000000.0)
p2 = Protein(id="p2", mass=1000000.0)
builder.add_proteins([p1, p2])

### Create and add metabolites:
### - External and internal glucose
### - One generic Protein product
### - Masses in Da.
x_G     = Metabolite(id="x_G", species_location=SpeciesLocation.EXTERNAL, mass=180.0)
G       = Metabolite(id="G", species_location=SpeciesLocation.INTERNAL, mass=180.0)
Protein = Metabolite(id="Protein", species_location=SpeciesLocation.INTERNAL,mass=180.0)
builder.add_metabolites([x_G, G, Protein])

### Create and add transporter to import glucose:
### - Enzyme is composed of one protein p1
### - Reaction is irreversible
### - kcat values in 1/h
### - KM values in g/L
rxn1 = Reaction(id="rxn1", lb=0.0, ub=1000.0,
                reaction_type=ReactionType.TRANSPORT,
                metabolites={"x_G":-1.0, "G": 1.0},
                proteins={"p1": 1.0})
rxn1.add_kcat_value(direction=ReactionDirection.FORWARD, kcat_value=45000.0)
rxn1.add_km_value(metabolite_id="x_G", km_value=0.00013)
rxn1.complete(kcat_value=0.0, km_value=0.0)
builder.add_reaction(rxn1)

### Create and add ribosome reaction to produce proteins:
### - Enzyme is composed of one protein p2
### - Reaction is irreversible
ribosome = Reaction(id="Ribosome", lb=0.0, ub=1000.0,
                    reaction_type=ReactionType.METABOLIC,
                    metabolites={"G":-1.0, "Protein": 1.0},
                    proteins={"p2": 1.0})
ribosome.add_kcat_value(direction=ReactionDirection.FORWARD, kcat_value=45000.0)
ribosome.add_km_value(metabolite_id="G", km_value=0.00013)
ribosome.complete(kcat_value=0.0, km_value=0.0)
builder.add_reaction(ribosome)

### Convert the model to GBA formalism (cf. Dourado et al. 2023)
builder.convert(ribosome_mass_kcat=4.55, ribosome_mass_km=8.3)
builder.build_GBA_model()

### Set cell's total density (g/L)
builder.set_rho(340.0)

### Create external conditions (in g/L)
x_G_conc = 1.0
for i in range(25):
    builder.add_condition(condition_id=str(i+1), metabolites={"x_G": x_G_conc})
    x_G_conc *= 2/3

### Save the model to an ODS file
builder.export_to_ods()

# Load the model and run optimization

In [2]:
from gba import read_ods_model

### Load the ODS model
model = read_ods_model(name="toy")

### Find a valid initial solution
model.find_initial_solution()

### Optimize the model for all conditions
model.find_optimum_by_condition()

### Make a plot
model.plot(x="x_G", y="mu", title="Growth rate", logx=True)

### Export optimization data in CSV
model.export_optimization_data()